In [17]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

In [18]:
df=pd.read_csv('houses.csv')
df.rename(columns={'Unnamed: 0':'id'},inplace=True) 
#df.head()

In [19]:
df.columns

Index(['id', 'listingUrl', 'listingId', 'listingTitle', 'listingCity',
       'listingTime', 'listingName', 'listingTel', 'listingAdd', 'listingMail',
       'listing_house_type', 'listing_room_type', 'listing_parking',
       'listing_facility', 'listing_rent_cover', 'listing_rent_type',
       'listing_independent_access', 'listing_rent_date', 'listing_rent_price',
       'listing_describe', 'data_time'],
      dtype='object')

## Data Wrangling

In [20]:
# View the number NaN in a column
# Fill in field listingTitle
print(df['listingCity'].isnull().sum(axis = 0))
dd=df[df['listingCity'].isnull()]
conditions=[dd['listingTitle'].str.contains('温西|温东|nanaimo天车|UBC|29天车站|29街天车|二十九街|langara'),dd['listingTitle'].str.contains('本拿比|Burnaby|Brentwood|Deer Lake|metrotown'),\
            dd['listingTitle'].str.contains('列治文'),dd['listingTitle'].str.contains('素里'),dd['listingTitle'].str.contains('枫树岭'), dd['listingTitle'].str.contains('高贵林|Burquitlam')]
choices=['Vancouver','Burnaby','Richmond','Surrey','Maple Ridge','Coquitlam']
dd['listingCity']=np.select(conditions,choices)
#print(dd[['id','listingTitle','listingCity']])
df['listingCity']=df['listingCity'].fillna(dd['listingCity'])
#Fill in empty listing_room_type (1762-568)
dd=df[df['listing_room_type'].isnull()]
conditions=[dd['listingTitle'].str.contains('1房|单间|单房|有个房间|一房|一卧房|朝南房间|大单房|套房|一室|房间|1b|卧室|主卧|次卧'),dd['listingTitle'].str.contains('两房|二房|2b|二室|2房|2室|2卧|两间|二卧|两室'),\
            dd['listingTitle'].str.contains('三房|3卧|3房|3b'),dd['listingTitle'].str.contains('4房|四房'),dd['listingTitle'].str.contains('5房')]
choices=['1睡房','2睡房','3睡房','4睡房','5睡房']
dd['listing_room_type']=np.select(conditions,choices)
df['listing_room_type']=df['listing_room_type'].fillna(dd['listing_room_type'])


71


C:\Users\myf\AppData\Local\Temp\ipykernel_39696\4075557303.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['listingCity']=np.select(conditions,choices)
C:\Users\myf\AppData\Local\Temp\ipykernel_39696\4075557303.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['listing_room_type']=np.select(conditions,choices)


In [21]:
print(df['listingCity'].isnull().sum(axis = 0))

0


## Add some new fields

In [22]:
# Create new DataFrame dd, add field 'room_nums', 'month', 'day'
dd=df[df['listing_room_type'].notnull()]
dd = df[df['listingTime'].notnull()]
dd['room_nums']=dd['listing_room_type'].str.split('；',expand=True)[0]
dd['mon']=dd['listingTime'].apply(lambda x:'-'.join(x.split('-')[0:2]))
dd['day']=dd['listingTime'].apply(lambda x:x.split(', ')[0])


C:\Users\myf\AppData\Local\Temp\ipykernel_39696\1905496371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['room_nums']=dd['listing_room_type'].str.split('；',expand=True)[0]
C:\Users\myf\AppData\Local\Temp\ipykernel_39696\1905496371.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['mon']=dd['listingTime'].apply(lambda x:'-'.join(x.split('-')[0:2]))
C:\Users\myf\AppData\Local\Temp\ipykernel_39696\1905496371.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [23]:
# Make DataFrame Pipline Interactive
idf=dd.interactive()

## Distribution of rental housing in each city

In [29]:
# https://hvplot.holoviz.org/
# https://panel.holoviz.org/user_guide/Widgets.html

In [24]:
# Radio buttons for month choice
month = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['2022-10','2022-11','2022-12'], 
    button_type='success'
)
month

RadioButtonGroup(button_type='success', name='Y axis', options=['2022-10', '2022-11', ...], value='2022-10')

In [25]:
dis_renting_pipeline = (idf[idf.listingTime.str.startswith(month).fillna(False)]['listingCity'].value_counts().sort_values(ascending=False))
dis_renting_pipeline
a=dis_renting_pipeline.hvplot(kind='bar',
                            title='Distribution of Rental Housing in Each City',
                            rot=30,
                           )

## Average price of housings distribution

In [26]:
city = pn.widgets.Select(name='City', options=['Vancouver', 'Burnaby', 'Richmond','Surrey','Coquitlam'])
city

Select(name='City', options=['Vancouver', ...], value='Vancouver')

In [27]:
avg_price_rooms_pipeline = (idf[(idf.listingCity==city) & (idf.mon.isin(['2022-10','2022-11']))]
                            .groupby(['room_nums','mon'])['listing_rent_price'].mean().round(2)
                            .to_frame()
                           )

avg_price_rooms_pipeline
b=avg_price_rooms_pipeline.hvplot(kind='bar',xlabel='Room Type',ylabel='Avg Price',stacked=True,
                            title='Average price of housings distribution',
                            )

## Daily comparison of new housing resources in different cities

In [28]:
rooms_nums_day_pipeline = (idf[(idf.listingTime.str.startswith(month).fillna(False)) & (idf.listingCity.isin(['Vancouver','Burnaby','Richmond','Surrey','Coquitlam']))]
                            .groupby(['listingCity','day'])['listingId'].count()
                            .to_frame()
                            .reset_index()
                            .sort_values(by='day')
                            .reset_index(drop=True)
                          )
rooms_nums_day_pipeline
c=rooms_nums_day_pipeline.hvplot(x='day', by='listingCity',line_width=2,title="Daily comparison of new housing resources in different cities", rot=90,ylabel='Houses Number')

## Houses Renting Type and Price

In [29]:
new_pipeline = (idf[(idf.listingCity==city) & (idf.listingTime.str.startswith(month).fillna(False)) & (idf.listing_rent_type.notnull)]
                            .groupby(['room_nums','listing_rent_type'])['listing_rent_price'].mean().round(2)
                            .to_frame()
                           )
new_pipeline 
d=new_pipeline.pipe(pn.widgets.Tabulator,pagination='remote', layout='fit_columns',height=300, page_size=10,sizing_mode='stretch_width')


## Creating Dashboard

In [30]:
#Layout using Template
template = pn.template.FastListTemplate(
    title = 'Greater Vancouver rental market',
    sidebar = [pn.pane.Markdown("# Greater Vancouver"),
               pn.pane.Markdown("#### Different City's Rental Market Change"),
               pn.pane.Markdown("#### Different Month's Rental Market Change"),
               pn.pane.Markdown("#### Different Room-Type's Rental Market Change"),
               pn.pane.PNG('rent.png',width=330),
               pn.pane.Markdown("## Settings"),
               month,
               city],
    main = [pn.Row(pn.Column(a.panel(width=600), margin=(0,25)), pn.Column(b.panel(width=600))),
            pn.Row(pn.Column(c.panel(width=700), margin=(0,25)), pn.Column(d.panel(width=500)))],
    accent_base_color="#2F4F4F",
    header_background="#2F4F4F",)
##88d8b0" cale_both
template.show() 
#template.servable();
    
    

Launching server at http://localhost:49221
